### Preparation - Review raw usage data

In [ ]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
from matplotlib import pyplot as plt
import datetime
import numpy as np
from plot_helper import *

CSV_FILE = '../data/GlobalUsageReport.csv'

try:
    df_raw
except NameError:
    df_raw = pd.read_csv(CSV_FILE,encoding="ISO-8859-1",parse_dates=['Usage Period'])

df_raw.groupby('Usage Period')['Value (USD)'].sum().plot()
format_trend_chart()
plt.show()



### Preparation - Configure QUARTER_RANGE in defines.py

In [ ]:
from defines import *

df = df_raw[(df_raw['Usage Period']<=LAST_QUARTER[2]) & (df_raw['Usage Period']>=PREV_QUARTER[0])]
df.groupby('Usage Period')['Value (USD)'].sum().plot()
format_trend_chart()
plt.title("Data Range, {} ~ {}".format(QUARTER_RANGE[0][0],QUARTER_RANGE[1][1]))
plt.show()



### Total VCPP Revenue & Parnter Cound Trend

In [ ]:
from functools import reduce

plt.close()
fig,ax1 = plt.subplots()
ax2 = ax1.twinx()

rev = df.groupby('Usage Period')['Value (USD)'].sum()
ax1.bar(rev.index,rev.values,10,label='Total Revenue')

cnt = df.groupby('Usage Period')['Service Provider'].unique()
pcnts = list(map(lambda x: len(x), cnt.values))
ax2.plot(cnt.index,pcnts,'o-',c='tomato',label='Partner Count')
ax2.set_ylim(bottom=3500,top=3500*rev.values.max()/rev.values.min())
ax2.set_ylabel('vCAN Partner Count')
ax2.legend(loc="lower right", bbox_to_anchor=(1,0.1))
    
x0=len(cnt.loc[PREV_QUARTER[2]])
x1=len(cnt.loc[LAST_QUARTER[2]])
print("YoY Partner Count Growth: +{:.0f}%".format((x1/x0-1)*100))

x0=reduce(lambda x,y:x+y,map(lambda x: rev.loc[x],PREV_QUARTER))
x1=reduce(lambda x,y:x+y,map(lambda x: rev.loc[x],LAST_QUARTER))
print("YoY Revenue Growth: +{:.0f}%".format((x1/x0-1)*100))

ax1.yaxis.grid(which="major", color='black', linestyle='dashed', linewidth=.3)
ax1.set_yticklabels(['${:,.0f}K'.format(x/1000) for x in ax1.get_yticks()])
ax1.set_ylabel('vCAN Total MRR')
ax1.legend(loc="lower right", bbox_to_anchor=(1,0))

fig = plt.gcf()
fig.autofmt_xdate() 
plt.title("vCAN Business Trend")
fig.set_size_inches(10,6)
plt.show()

### Showcase Partner Monthly Revenue Trend

In [ ]:
showcase_partners = df[(df['Usage Period'] == LAST_QUARTER[2]) &
                       (df['Partner Status'] == 'Showcase')].groupby('Partner Group Name')['Value (USD)'].sum().sort_values(ascending=False).index

for partner in showcase_partners:
    partner_rev = df[df['Partner Group Name'] == partner].groupby('Usage Period')['Value (USD)'].sum()
    plt.plot(partner_rev.index,partner_rev.values,'o-',label=partner)

format_trend_chart()

plt.title("Showcase Partners Monthly Revenue Trend")
plt.show()

### Showcase Partner Growth Trend

In [ ]:
def plot_rev_trend(data,lbl):
#    plt.plot(data.index,(data.values/data.values[0]-1)*100,'o-',label=lbl,linewidth=5.0,markersize=9.0)
    plt.plot(data.index,(data.values/data.values[0]-1)*100,'o-',label=lbl)
        
for d in [
            [df.groupby('Usage Period')['Value (USD)'].sum(),'vCAN Total'],
            [df[df['Partner Status']=='Showcase'].groupby('Usage Period')['Value (USD)'].sum(),'Showcase Total'],
            [df[df['Partner Group Name']=='Fujitsu'].groupby('Usage Period')['Value (USD)'].sum(),'Fujitsu Group Total'],
            [df[df['Partner Group Name']=='NTT'].groupby('Usage Period')['Value (USD)'].sum(),'NTT Group Total']
        ]:
    plot_rev_trend(d[0],d[1])
 
ax = format_trend_chart()
ax.set_yticklabels(['+{:,.0f}%'.format(x) for x in ax.get_yticks()])
 #   ax.legend(loc="lower right", bbox_to_anchor=(1,0))
plt.show()


### VCPP Partners Revenue Distribution

In [ ]:
def plot_hist(data_prev,data_last,cutoff,title):
    plt.hist(data_last[:int(len(data_last)*cutoff)],bins=100,alpha=.8,label='2017 Q1')
    plt.hist(data_prev[:int(len(data_last)*cutoff)],bins=100,alpha=.8,label='2016 Q1')

    plt.xlabel("Quarterly Revenue (USD)")
    plt.ylabel("Number of Service Providers")
    plt.title(title)
    ax = plt.gca()
    fig = plt.gcf()
    ax.set_xticklabels(['${:,.0f}K'.format(x/1000) for x in ax.get_xticks()])
    ax.legend(loc="upper right")
    fig.set_size_inches(10,6)
    plt.show()
    
df_last_q = df[(df['Usage Period'] >= LAST_QUARTER[0]) & (df['Usage Period'] <= LAST_QUARTER[2])]
df_prev_q = df[(df['Usage Period'] >= PREV_QUARTER[0]) & (df['Usage Period'] <= PREV_QUARTER[2])]
data_last = df_last_q.groupby('Service Provider')['Value (USD)'].sum().sort_values(ascending=False)
data_prev = df_prev_q.groupby('Service Provider')['Value (USD)'].sum().sort_values(ascending=False)
    
plot_hist(data_prev,data_last,1.0,"Revenue Distribution - All SPs")
plot_hist(data_prev,data_last,0.1,"Revenue Distribution - Top 10% SPs")
    
total_last = df_last_q['Value (USD)'].sum()
print("Total SP Count: {}".format(len(data_last)))
print("Top {} SPs Contribution: {:.0f}%".format(10,data_last[:10].sum()/total_last*100))
    
plt.pie(data_last.values,startangle=90,counterclock=False)
plt.title('Revenue Distribution')
plt.gcf().set_size_inches(3,3)
plt.show()


### Showcase Partner Group Companies Revenue Trend

In [ ]:
def partner_rev_trend(group):
    # sort by revenue size at the last month
    df_lm = df[df['Usage Period'] == LAST_QUARTER[2]]

    s_sp = df_lm[df_lm['Partner Group Name'] == group].groupby('Service Provider')['Value (USD)'].sum().sort_values(ascending=False)

    for sp in s_sp.index:
        s_sprev = df[df['Service Provider'] == sp].groupby('Usage Period')['Value (USD)'].sum()
    
        # populate 0s for all months to capture no-report months
#        for i in up_index:
#            if not i in partner_rev.keys():
#                partner_rev.set_value(i,0.0)
#                data = partner_rev.sort_index()
    
        plt.plot(s_sprev.index,s_sprev.values,'o-',label=sp)

    format_trend_chart()

    plt.title('{} Group Revenue Trend Chart'.format(group))
    plt.show()
    
partner_rev_trend('NTT')
partner_rev_trend('Fujitsu')


### Revenue Size vs Growth

In [ ]:
import size_vs_growth as sg

sg.plot(df,
     {'type':['GEO','RANK'],
      'time_frame': 'LAST_12',
      'min_rev_growth':10000,
      'show_sp_name':500000})
    
sg.plot(df[(df['Country']=='Japan')],
    {'type':['NO_SLICE'],
     'time_frame': 'LAST_12',     
     'min_rev_growth':10000,
    'show_sp_name':100000})